In [1]:
import os
import cv2
import numpy as np
from imutils import paths

from sklearn.preprocessing import LabelBinarizer

In [2]:
datapath = 'Data'
outputmodel = 'Video_classification_model'
outputlabelbinarizer = 'v_c_binarizer'

In [3]:
video_labels = set(['boxing','swimming','table_tennis'])
print('Image is being Loaded...')

pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label in video_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)

Image is being Loaded...


In [4]:
data = np.array(data)
labels = np.array(labels)

#hot encoded value 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels,test_size=0.2, stratify=labels, random_state=42)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
trainGenerator = ImageDataGenerator(
        rotation_range = 30,
        zoom_range = 0.15,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.15,
        horizontal_flip = True,
        fill_mode = 'nearest')

validationGenerator = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype='float32')
trainGenerator.mean = mean
validationGenerator.mean = mean

In [9]:
from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout
from keras.models import Model

In [10]:
weights = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
baseModel = ResNet50(weights= weights, include_top=False, input_tensor=Input(shape=(224,224,3)))

In [11]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)

headModel = Dense(len(lb.classes_), activation='softmax')(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

In [12]:
from keras.optimizers import SGD

In [16]:
epoch = 3
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [17]:
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['acc'])

In [18]:
History = model.fit_generator(
            trainGenerator.flow(X_train, Y_train, batch_size =32),
            steps_per_epoch = len(X_train) // 32,
            validation_data = validationGenerator.flow(X_test, Y_test),
            validation_steps = len(X_test) // 32,
            epochs = epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
306/306 [==============================] - 2160s 7s/step - loss: 3.1004 - acc: 0.1338 - val_loss: 2.2220 - val_acc: 0.3775
Epoch 2/3
306/306 [==============================] - 1954s 6s/step - loss: 2.3809 - acc: 0.2866 - val_loss: 1.8525 - val_acc: 0.5082
Epoch 3/3
306/306 [==============================] - 1845s 6s/step - loss: 2.1069 - acc: 0.3762 - val_loss: 1.6205 - val_acc: 0.5781


In [19]:
import pickle

In [31]:
model.save('outputmodel.h5')
lbinarizer = open(r'v_c_binarizer.pickle','wb')
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()